In [ ]:
%matplotlib widget
import scipp as sc
import plopp as pp

from ess.estia.load import load_mcstas_events
from ess.estia.calibration import reflectivity_provider
from ess.estia import EstiaWorkflow
from ess.reflectometry.types import *
from ess.reflectometry.supermirror import CriticalEdge


wf = EstiaWorkflow()
wf.insert(load_mcstas_events)
wf.insert(reflectivity_provider)

wf[YIndexLimits]  = sc.scalar(35), sc.scalar(64)
wf[ZIndexLimits] = sc.scalar(0), sc.scalar(14 * 32)
wf[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')
wf[WavelengthBins] = sc.geomspace('wavelength', 3.5, 12, 2001, unit='angstrom')
wf[QBins] = sc.linspace('Q', 0.001, 0.1, 200, unit='1/angstrom')
# Reference sample is perfect supermirror with reflectivity = 1 everywhere
wf[CriticalEdge] = sc.scalar(float('inf'), unit='1/angstrom')

# There is no proton current data in the McStas files, here we just add some fake proton current
# data to make the workflow run.
wf[ProtonCurrent[SampleRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})
wf[ProtonCurrent[ReferenceRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})

In [ ]:
def reduce_as_sample(sample_run, reference_run):
    w = wf.copy()
    w[Filename[SampleRun]] = path(sample_run)
    w[Filename[ReferenceRun]] = path(reference_run)
    return w.compute(Sample)

def reduce_as_reference(sample_run, reference_run):
    w = wf.copy()
    w[Filename[SampleRun]] = path(sample_run)
    w[Filename[ReferenceRun]] = path(reference_run)
    r = w.compute(Reference)
    _unalign_coords(r)
    return r

def _unalign_coords(da):
    for c in da.coords:
        da.coords.set_aligned(c, False)

def path(num):
    return f'results/{num}/tof_detector_moved_list.p.x.y.t.L.sx.sy.sz.U1'


In [ ]:
run_numbers = dict(  # noqa: C408
    reference=list(range(221385, 221385+4)),
    magnetic_reference=list(range(221385+4, 221385+8)),
    magnetic_reference_short=list(range(221229+4, 221229+8)),
    magnetic_reference01=list(range(221502, 221502+4)),
    magnetic_reference03=list(range(221502+4, 221502+8)),
    magnetic_reference05=list(range(221510, 221514)),
    magnetic_reference07=list(range(221514, 221518)),
    magnetic_test_sample=list(range(221421, 221425)),
    magnetic_reference001=list(range(221536, 221540)),
    spin_flip_sample_short=list(range(221577, 221581)),
    spin_flip_sample=list(range(221585, 221589)),
)

In [ ]:

from ess.estia.types import *

results = {}
for run in (
    'reference',
    'magnetic_reference',
    #'magnetic_reference_short',
    'magnetic_reference01',
    #'magnetic_reference03', 'magnetic_reference05', 'magnetic_reference07', 'magnetic_reference001'
):
    results[run] = []
    for num in run_numbers[run]:
        results[run].append(reduce_as_reference(run_numbers['magnetic_test_sample'][0], num))
        print(run, path(run))


In [ ]:

for name, run in (
    ('magnetic_sample', 'magnetic_reference'),
    #('magnetic_sample_short', 'magnetic_reference_short'),
    #('magnetic_sample01', 'magnetic_reference01'),
    #('magnetic_sample03', 'magnetic_reference03'),
    #('magnetic_sample05', 'magnetic_reference05'),
    #('magnetic_sample07', 'magnetic_reference07'),
    #('magnetic_test_sample', 'magnetic_test_sample'),
    ('spin_flip_sample', 'spin_flip_sample'),
    #('spin_flip_sample_short', 'spin_flip_sample_short'),
):
    results[name] = []
    for num in run_numbers[run]:
        results[name].append(reduce_as_sample(num, run_numbers['reference'][0]))
        print(name, run, path(num))




In [ ]:

def read_supermirror_reflectivity_file(fname):
    with open(fname) as f:
        d = [v.strip().split('\t') for v in f.readlines()][:-1]

    return sc.DataArray(data=sc.array(dims='Q',  values=[float(v[1]) for v in d]), coords={'Q': sc.array(dims='Q', values=[float(v[0]) for v in d], unit='1/angstrom')})


Rdown = read_supermirror_reflectivity_file('SNAG_m5p5_Tdown.ref')
Rup = read_supermirror_reflectivity_file('SNAG_m5p5_Tup.ref')


# Results with polarization correction

## Spin flip sample: magnetic supermirror with 10% spin flip probability

In [ ]:
magres = {}
magnetic_reference = 'magnetic_reference'
sample = 'spin_flip_sample'

w = wf.copy()
w.insert(reflectivity_provider)
w[Intensity[NonMagneticReference, OffOff]] = results['reference'][0]
w[Intensity[NonMagneticReference, OffOn]] = results['reference'][1]
w[Intensity[NonMagneticReference, OnOff]] = results['reference'][2]
w[Intensity[NonMagneticReference, OnOn]] = results['reference'][3]

w[Intensity[MagneticReference, OffOff]] = results[magnetic_reference][0]
w[Intensity[MagneticReference, OffOn]] = results[magnetic_reference][1]
w[Intensity[MagneticReference, OnOff]] = results[magnetic_reference][2]
w[Intensity[MagneticReference, OnOn]] = results[magnetic_reference][3]

w[Intensity[MagneticSample, OffOff]] = results[sample][0]
w[Intensity[MagneticSample, OffOn]] = results[sample][1]
w[Intensity[MagneticSample, OnOff]] = results[sample][2]
w[Intensity[MagneticSample, OnOn]] = results[sample][3]
R = w.compute(PolarizedReflectivityOverQ)

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': sc.abs(r) for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown * 9/10, 'True $R_{\\uparrow\\uparrow}$': Rup * 9/10},
    title=f'Reflectivities, corrected, {sample}',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p

## Magnetic supermirror sample - 0% spin flip probability

In [ ]:
magres = {}
magnetic_reference = 'magnetic_reference01'
sample = 'magnetic_sample'

w = wf.copy()
w.insert(reflectivity_provider)
w[Intensity[NonMagneticReference, OffOff]] = results['reference'][0]
w[Intensity[NonMagneticReference, OffOn]] = results['reference'][1]
w[Intensity[NonMagneticReference, OnOff]] = results['reference'][2]
w[Intensity[NonMagneticReference, OnOn]] = results['reference'][3]

w[Intensity[MagneticReference, OffOff]] = results[magnetic_reference][0]
w[Intensity[MagneticReference, OffOn]] = results[magnetic_reference][1]
w[Intensity[MagneticReference, OnOff]] = results[magnetic_reference][2]
w[Intensity[MagneticReference, OnOn]] = results[magnetic_reference][3]

w[Intensity[MagneticSample, OffOff]] = results[sample][0]
w[Intensity[MagneticSample, OffOn]] = results[sample][1]
w[Intensity[MagneticSample, OnOff]] = results[sample][2]
w[Intensity[MagneticSample, OnOn]] = results[sample][3]
R = w.compute(PolarizedReflectivityOverQ)

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': sc.abs(r) for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown, 'True $R_{\\uparrow\\uparrow}$': Rup},
    title=f'Reflectivities, corrected, {sample}',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p

# Results without polarization correction

## Spin flip sample: magnetic supermirror with 10% spin flip probability

In [ ]:
from ess.estia.calibration import solve_for_calibration_parameters
from ess.reflectometry.normalization import reduce_to_q
I0, *_ = solve_for_calibration_parameters(
    [reduce_to_q(r, wf.compute(QBins)) for r in results['reference']],
    [reduce_to_q(r, wf.compute(QBins)) for r in results['magnetic_reference']]
)

R = [
    reduce_to_q(v, wf.compute(QBins)).hist() / sc.values(I0) for v in results['spin_flip_sample']
]

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': sc.abs(r) for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown * 9/10, 'True $R_{\\uparrow\\uparrow}$': Rup * 9/10},
    title=f'Reflectivities, not corrected, {sample}',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p

## Magnetic supermirror sample - 0% spin flip probability

In [ ]:
from ess.estia.calibration import solve_for_calibration_parameters
from ess.reflectometry.normalization import reduce_to_q
I0, *_ = solve_for_calibration_parameters(
    [reduce_to_q(r, wf.compute(QBins)) for r in results['reference']],
    [reduce_to_q(r, wf.compute(QBins)) for r in results['magnetic_reference01']]
)

R = [
    reduce_to_q(v, wf.compute(QBins)).hist() / sc.values(I0) for v in results['magnetic_sample']
]

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']

p = pp.plot(
    {
        f'{labels[i]}': sc.abs(r) for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown, 'True $R_{\\uparrow\\uparrow}$': Rup},
    title=f'Reflectivities, not corrected, {sample}',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p